In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Welcome to JupyROOT 6.29/01
Recompile is not needed


'Utility Functions V1.0.4 \n'

In [2]:
ch = R.TChain("MiniDST")
data_mod = "MC"
data_source="/data/HPS/data/physrun2021/" + data_mod
energy="3.0"
particle="e-"
if particle == "e+":
    particle_s = "e\+"
else:
    particle_s = particle
if energy == "*":
    energy_s = "all_"
else:
    energy_s = energy;
ch.Add(data_source+"/hpsForward_"+particle_s+"_"+energy+"GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")
df = R.RDataFrame(ch)

Number of events loaded:   1.243M


In [3]:
dfx = df.Define("n_kf_track","int n=0; for(int i=0;i<track_type.size();++i){if(track_type[i]==1) n+=1;}; return n") \
    .Filter("n_kf_track==1").Filter("ecal_cluster_energy.size()==1") \
    .Define("trk_idx","for(int i=0;i<track_type.size();++i){if(track_type[i]==1) return(i);} return(-1);") \
    .Define("diff_posx_ecal","ecal_cluster_x[0] - track_x_at_ecal[trk_idx]") \
    .Define("diff_posy_ecal","ecal_cluster_y[0] - track_y_at_ecal[trk_idx]") \
    .Define("diff_E_p","part_energy[0] - get_vec_abs(part_px,part_py,part_pz)[0]") \
    .Define("diff_p_kf_gbl","get_vec_abs(part_px,part_py,part_pz)[0] - get_vec_abs(part_px,part_py,part_pz)[1]") \
    .Define("mc_part_primary_z","find_end_z_of_primary_mc_part(mc_part_z, mc_part_end_z)") \
    .Define("mc_part_daughters_z","find_average_end_of_daughters_of_primary_mc_part(mc_part_z, mc_part_end_z, mc_part_daughters)")
cnt  = df.Count()
cntx = dfx.Count()

In [4]:
hx_diff_posx_ecal=dfx.Histo1D(("hx_diff_posx","Diff Ecal_x - Track_x;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_top=dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_posx_top","Diff Ecal_x - Track_x Top;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_bot=dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_posx_bot","Diff Ecal_x - Track_x Bot;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_L=dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_posx_L","Diff Ecal_x - Track_x L;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_R=dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_posx_R","Diff Ecal_x - Track_x R;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posy_ecal=dfx.Histo1D(("hx_diff_posy","Diff Ecal_y - Track_y;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_top=dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_posy_top","Diff Ecal_y - Track_y Top;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_bot=dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_posy_bot","Diff Ecal_y - Track_y Bot;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_L=dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_posy_L","Diff Ecal_y - Track_y L;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_R=dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_posy_R","Diff Ecal_y - Track_y R;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_e_p = dfx.Histo1D(("hx_diff_e_p", "Diff Energy - Momentum;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_top = dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_e_p_top", "Diff Energy - Momentum Top;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_bot = dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_e_p_bot", "Diff Energy - Momentum Bot;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_L = dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_e_p_L", "Diff Energy - Momentum Left;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_R = dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_e_p_R", "Diff Energy - Momentum Right;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_p_kf_gbl = dfx.Histo1D(("hx_diff_p_kf_gbl","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_top = dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_p_kf_gbl_top","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_bot = dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_p_kf_gbl_bot","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_L = dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_p_kf_gbl_L","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_R = dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_p_kf_gbl_R","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")

hx_mc_part_primary_z = dfx.Histo1D(("hx_mc_part_primary_z","End of track for primary MC particle.;z[mm]",1000,-0.1,1700.),"mc_part_primary_z")
hx_mc_part_ave_daughter_z = dfx.Histo1D(("hx_mc_part_ave_daughter_z","Average end of track for primary MC particle daughters.;z[mm]",1000,-0.1,1700.),"mc_part_daughters_z")


In [5]:
cc4 = R.TCanvas("cc4","Canvas",1200,600)
cc4.Divide(2,2)
p1=cc4.cd(1)
#p1.SetLogy()
hx_diff_posx_ecal.Draw()
hx_diff_posx_ecal_top.SetLineColor(R.kGreen)
hx_diff_posx_ecal_top.Draw("same")
hx_diff_posx_ecal_bot.SetLineColor(R.kRed)
hx_diff_posx_ecal_bot.Draw("same")
hx_diff_posx_ecal_L.SetLineColor(R.kMagenta)
hx_diff_posx_ecal_L.Draw("same")
hx_diff_posx_ecal_R.SetLineColor(R.kCyan)
hx_diff_posx_ecal_R.Draw("same")
p2=cc4.cd(2)
#p2.SetLogy()
hx_diff_posy_ecal.Draw()
hx_diff_posy_ecal_top.SetLineColor(R.kGreen)
hx_diff_posy_ecal_top.Draw("same")
hx_diff_posy_ecal_bot.SetLineColor(R.kRed)
hx_diff_posy_ecal_bot.Draw("same")
hx_diff_posy_ecal_L.SetLineColor(R.kMagenta)
hx_diff_posy_ecal_L.Draw("same")
hx_diff_posy_ecal_R.SetLineColor(R.kCyan)
hx_diff_posy_ecal_R.Draw("same")
p3=cc4.cd(3)
# p3.SetLogy()
hx_diff_e_p.Draw()
hx_diff_e_p_top.SetLineColor(R.kGreen)
hx_diff_e_p_top.Draw("same")
hx_diff_e_p_bot.SetLineColor(R.kRed)
hx_diff_e_p_bot.Draw("same")
hx_diff_e_p_L.SetLineColor(R.kMagenta)
hx_diff_e_p_L.Draw("same")
hx_diff_e_p_R.SetLineColor(R.kCyan)
hx_diff_e_p_R.Draw("same")
p4=cc4.cd(4)
# p4.SetLogy()
hx_diff_p_kf_gbl.Draw()
hx_diff_p_kf_gbl_top.SetLineColor(R.kGreen)
hx_diff_p_kf_gbl_top.Draw("same")
hx_diff_p_kf_gbl_bot.SetLineColor(R.kRed)
hx_diff_p_kf_gbl_bot.Draw("same")
hx_diff_p_kf_gbl_L.SetLineColor(R.kMagenta)
hx_diff_p_kf_gbl_L.Draw("same")
hx_diff_p_kf_gbl_R.SetLineColor(R.kCyan)
hx_diff_p_kf_gbl_R.Draw("same")

# h_trk_type.Draw()
cc4.cd(0)
leg1 = R.TLegend(0.83,0.8,1.,1.)
leg1.AddEntry(hx_diff_posx_ecal.GetValue(), "All data with one cluster")
leg1.AddEntry(hx_diff_posx_ecal_top.GetValue(),"Top only")
leg1.AddEntry(hx_diff_posx_ecal_bot.GetValue(),"Bottom only")
leg1.AddEntry(hx_diff_posx_ecal_L.GetValue(),"Left Only x<0")
leg1.AddEntry(hx_diff_posx_ecal_R.GetValue(),"Right Only x>0")
leg1.Draw()

cc4.Draw()
cc4.SaveAs(data_mod+"_"+particle+"_"+energy_s+"GeV_compare.pdf")

Info in <TCanvas::Print>: pdf file MC_e-_3.0GeV_compare.pdf has been created


In [6]:
cc5 = R.TCanvas("cc5","Canvas",800,600)
hx_mc_part_ave_daughter_z.SetStats(0)
hx_mc_part_ave_daughter_z.SetLineColor(R.kRed)
hx_mc_part_ave_daughter_z.SetLineWidth(2)
hx_mc_part_ave_daughter_z.Draw()
l1 = R.TLine(1394, 0, 1394, hx_mc_part_ave_daughter_z.GetMaximum())
l1.SetLineWidth(2)
l1.SetLineColor(R.kOrange)

l1.Draw()
hx_mc_part_primary_z.SetLineColor(R.kBlue)
hx_mc_part_primary_z.Draw("same")
leg5 = R.TLegend(0.1,0.7,0.4,0.9)
leg5.AddEntry(hx_mc_part_primary_z.GetValue(),"Endpoint of primary MC particle")
leg5.AddEntry(hx_mc_part_ave_daughter_z.GetValue(),"Average endpoint of daughters.")
leg5.AddEntry(l1,"Ecal face.")
leg5.Draw()
cc5.Draw()

In [7]:
# Link the C++ fit_gaus_tail function to a TF1 so the fitting of the histogram works properly in Python
R.gInterpreter.Declare('''
void setup_gaus_tail(void){
auto gaustail = new TF1("gaustail",fit_gaus_tail,-20.,20.,4);
gaustail->SetParameters(1208, -5, 3.4, 1.);
gaustail->SetParNames("A","Mu","Area","tail");
std::cout << "The gaustail function is now setup. " << std::endl;
}
''')
R.setup_gaus_tail()

# Link the C++ fit_gaus_tail function to a TF1 so the fitting of the histogram works properly in Python
R.gInterpreter.Declare('''
void setup_gaus_tailn(void){
auto gaustailn = new TF1("gaustailn",fit_gaus_tailn,-20.,20.,4);
gaustailn->SetParameters(1208, -5, 3.4, 1.);
gaustailn->SetParNames("A","Mu","Area","tail");
std::cout << "The gaustailn function is now setup. " << std::endl;
}
''')
R.setup_gaus_tailn()

The gaustail function is now setup. 
The gaustailn function is now setup. 


In [18]:
cc6 = R.TCanvas("cc6","Canvas",1200,400)
cc6.Divide(2,1)
p1=cc6.cd(1)
#p1.SetLogy()
hx_diff_posx_ecal_L.SetLineColor(R.kMagenta)
hx_diff_posx_ecal_R.SetLineColor(R.kCyan)
hx_diff_posx_ecal_R.Draw()
hx_diff_posx_ecal_L.Draw("same")
print("Fit results for the 'left' distribution (magenta)")
R.gaustailn.SetParameter(0, 13400)
R.gaustailn.SetParameter(1, -4.19)
R.gaustailn.SetParameter(2,  3.4)
R.gaustailn.SetParameter(3,  0.01)
fit_result_L = hx_diff_posx_ecal_L.Fit("gaustailn", "S")
print("Fit results for the 'right' distribution (cyan)")
R.gaustail.SetParameter(1, 3.5)
fit_result_R = hx_diff_posx_ecal_R.Fit("gaus", "S")
p2=cc6.cd(2)
#p2.SetLogy()
tophist = hx_diff_posy_ecal_top.GetPtr()
xax = tophist.GetXaxis()
xax.SetLimits(-10,10)
hx_diff_posy_ecal_top.SetLineColor(R.kGreen)
hx_diff_posy_ecal_top.Draw()
hx_diff_posy_ecal_bot.SetLineColor(R.kRed)
hx_diff_posy_ecal_bot.Draw("same")
print("Fit results for the 'top' distribution (green)")
R.gaustail.SetParameter(1, 2)
fit_result_T = hx_diff_posy_ecal_top.Fit("gaus", "S")
print("Fit results for the 'bottom' distribution (red)")
R.gaustail.SetParameter(1, -1)
fit_result_B = hx_diff_posy_ecal_bot.Fit("gaus", "S")

# h_trk_type.Draw()
cc6.cd(0)
leg1 = R.TLegend(0.83,0.8,1.,1.)
leg1.AddEntry(hx_diff_posx_ecal.GetValue(), "All data with one cluster")
leg1.AddEntry(hx_diff_posx_ecal_top.GetValue(),"Top only")
leg1.AddEntry(hx_diff_posx_ecal_bot.GetValue(),"Bottom only")
leg1.AddEntry(hx_diff_posx_ecal_L.GetValue(),"Left Only x<0")
leg1.AddEntry(hx_diff_posx_ecal_R.GetValue(),"Right Only x>0")
leg1.Draw()

cc6.Draw()

Fit results for the 'left' distribution (magenta)
Fit results for the 'right' distribution (cyan)
Fit results for the 'top' distribution (green)
Fit results for the 'bottom' distribution (red)
 FCN=86857.1 FROM MIGRAD    STATUS=CONVERGED     405 CALLS         406 TOTAL
                     EDM=1.33931e-10    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   0.3 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            1.18335e+05   6.89002e+01   1.33043e-01   2.65243e-07
   2  Mu          -1.71821e+00   3.80916e-03   4.66248e-07   8.65098e-04
   3  Area         2.56035e+00   2.81619e-03  -1.16801e-06   6.76542e-03
   4  tail         3.65110e-01   5.63436e-04  -1.43140e-07  -1.40835e-02
 FCN=62451.4 FROM MIGRAD    STATUS=CONVERGED      75 CALLS          76 TOTAL
                     EDM=1.14165e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                             

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc6


In [19]:
R.gaustail.GetParameter(1)

-1.0

In [23]:
print(f"top_mean = [{fit_result_T.Parameter(1)}]")
print(f"top_mean_e = [{fit_result_T.Error(1)}]")
print(f"bot_mean = [{fit_result_B.Parameter(1)}]")
print(f"bot_mean_e = [{fit_result_B.Error(1)}]")
print(f"left_mean = [{fit_result_L.Parameter(1)}]")
print(f"left_mean_e = [{fit_result_L.Error(1)}]")
print(f"right_mean = [{fit_result_R.Parameter(1)}]")
print(f"right_mean_e = [{fit_result_R.Error(1)}]")

top_mean = [0.8491034824579542]
top_mean_e = [0.0005465571955235653]
bot_mean = [-0.675077954037954]
bot_mean_e = [0.0010992656875469878]
left_mean = [-1.7182140998711766]
left_mean_e = [0.0038091590937876793]
right_mean = [3.4865832364916374]
right_mean_e = [0.00171205258079781]
